In [ ]:
# Importar la biblioteca pandas
import pandas as pd

df = pd.read_csv(r'C:\\Users\\kevin\\Downloads\\archive\\rows.csv')

print(df.head())


In [3]:
df1 = df.copy()

In [ ]:
# 1. Limpiar los datos de nulos y datos incoherentes (incluyendo fechas)

date_format = '%d/%m/%Y'

df1['fecha reporte web'] = pd.to_datetime(df1['fecha reporte web'], format=date_format, errors='coerce')
df1['Fecha de notificación'] = pd.to_datetime(df1['Fecha de notificación'], format=date_format, errors='coerce')
df1['Fecha de inicio de síntomas'] = pd.to_datetime(df1['Fecha de inicio de síntomas'], format=date_format, errors='coerce')
df1['Fecha de muerte'] = pd.to_datetime(df1['Fecha de muerte'], format=date_format, errors='coerce')
df1['Fecha de diagnóstico'] = pd.to_datetime(df1['Fecha de diagnóstico'], format=date_format, errors='coerce')
df1['Fecha de recuperación'] = pd.to_datetime(df1['Fecha de recuperación'], format=date_format, errors='coerce')

df1 = df1.fillna('No registra')

df1.head()

In [ ]:
# 2. Hacer un consolidado de casos por departamento

consolidado_departamento = df1.groupby('Nombre departamento').size().reset_index(name='Total de casos')

display(consolidado_departamento)

In [ ]:
# 3.	Hacer un consolidado de casos por departamento  - municipio

consolidado_departamento_municipio = df1.groupby(['Nombre departamento', 'Nombre municipio']).size().reset_index(name='Total de casos')

display(consolidado_departamento_municipio)


In [ ]:
# 4.	Filtrar solo los casos de Risaralda

df1['Nombre departamento'] = df1['Nombre departamento'].str.strip().str.lower()

casos_risaralda = df1[df1['Nombre departamento'] == 'risaralda']

display(casos_risaralda)


In [ ]:
# 5.	Consolidar por Infancia – Adolescente -Adulto – Adulto Mayor teniendo en cuenta (en Risaralda)
  # a.	Infancia 0-14 años (No inclusive 14)
  # b.	Adolescente 14 años – 18 años (No inclusive 18)
  # c.	Adulto 18 años a 65 años (No inclusive 65)
  # d.	Adulto Mayor (65 años en adelante)

casos_risaralda = df1[df1['Nombre departamento'].str.strip().str.lower() == 'risaralda'].copy()

def clasificar_edad(edad):
    if isinstance(edad, (int, float)):  
        if 0 <= edad < 14:
            return 'Infancia'
        elif 14 <= edad < 18:
            return 'Adolescente'
        elif 18 <= edad < 65:
            return 'Adulto'
        elif edad >= 65:
            return 'Adulto Mayor'
    return 'No clasificado'  


casos_risaralda['Grupo Edad'] = casos_risaralda['Edad'].apply(clasificar_edad)


consolidado_edad = casos_risaralda.groupby('Grupo Edad').size().reset_index(name='Total de casos')


display(consolidado_edad)




In [ ]:
# 6.	Cuantas mujeres y hombres fueron contagiados de covid en Risaralda?

contagios_genero = casos_risaralda['Sexo'].value_counts().reset_index()
contagios_genero.columns = ['Sexo', 'Total de casos']

print("\nContagios de COVID en Risaralda por género:")
display(contagios_genero)

In [ ]:
# 7.	Cuantas mujeres y hombres murieron por covid en Risaralda?

muertes_genero = casos_risaralda[casos_risaralda['Recuperado'] == 'Fallecido']['Sexo'].value_counts().reset_index()
muertes_genero.columns = ['Sexo', 'Total de muertes']

display(muertes_genero)


In [ ]:
# 8.	Cuantos infantes, adolescentes, adultos y adultos mayores murieron por covid en Risaralda.

muertes_grupo_edad = casos_risaralda[casos_risaralda['Recuperado'] == 'Fallecido']['Grupo Edad'].value_counts().reset_index()
muertes_grupo_edad.columns = ['Grupo Edad', 'Total de muertes']

display(muertes_grupo_edad)


In [ ]:
# 9.	Cuanto tiempo en promedio por rango (igual que el punto anterior) duraron en recuperarse las personas por covid?

casos_risaralda['Fecha de recuperación'] = pd.to_datetime(casos_risaralda['Fecha de recuperación'], errors='coerce')
casos_risaralda['Fecha de inicio de síntomas'] = pd.to_datetime(casos_risaralda['Fecha de inicio de síntomas'], errors='coerce')

casos_risaralda['Tiempo recuperación (días)'] = (casos_risaralda['Fecha de recuperación'] - casos_risaralda['Fecha de inicio de síntomas']).dt.days

tiempo_recuperacion_promedio = casos_risaralda.groupby('Grupo Edad')['Tiempo recuperación (días)'].mean().reset_index()
tiempo_recuperacion_promedio.columns = ['Grupo Edad', 'Promedio de días para recuperarse']

display(tiempo_recuperacion_promedio)

In [ ]:
# 10.	Sacar el número de casos por municipio

casos_municipio = casos_risaralda['Nombre municipio'].value_counts().reset_index()
casos_municipio.columns = ['Municipio', 'Total de casos']

display(casos_municipio)